# Transformer

http://nlp.seas.harvard.edu/2018/04/03/attention.html

<img src="http://nlp.seas.harvard.edu/images/the-annotated-transformer_14_0.png" width="30%" />

용어 정리. 

- N번 반복되는 각 layer
- 그 안에 residual connection + layer norm + Sublayer(ff or attn) = sub-layer.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
# from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [2]:
torch.__version__

'0.4.1.post2'

In [3]:
device = torch.device('cuda:0')

In [4]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    # Q. src_mask 가 뭘까?
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

## Encoder

In [5]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [6]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [7]:
# Sublayer 를 wrapping 하는 sub-layer module.
# forward 시 Sublayer 를 인자로 받아서 그걸 내보내고, layer norm / dropout / residual connection 적용.
# 특이한 점이 하나 있는데, 그림과는 달리 LN => Layer => res-con 형태. 
# 때문에 layer 를 스택하는 Encoder/Decoder module 을 보면 마지막에 LN 을 한번 더 붙여준다.
# 다만 이렇게 되면 논문과는 달리 제일 앞에 LN 이 추가되게 되는데... 별로 상관 없으려나. 왜 굳이 이렇게 하는지 잘 모르겠다.
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [8]:
# layer 가 하나의 블록에 해당.
# 각 layer 가 N(=6) 번 스택되어 인코더를 이룬다.
# 위에서 언급했듯 sub-layer 가 norm-first 구조라서 최종 결과에는 한번 더 LayerNorm 을 씌워서 결과로 내보냄.
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [9]:
# Encoder block.
# self-attn layer + feed-forward layer
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # self-attention
        return self.sublayer[1](x, self.feed_forward) # feed-forward

## Decoder

In [10]:
# Encoder 와 마찬가지로 layer 를 받아서 N(=6) 번 스택.
# Encoder 와의 차이는 encoder hidden state 인 memory 와,
# 마스크가 소스/타겟으로 나뉜다.
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [11]:
# self-attn, enc-dec attn, ff 세가지 sub-layer 를 받아서 하나의 layer 를 만든다.
# 첫번째 sub-layer 인 self-attn 에서는 target mask 가,
# 두번째 sub-layer 인 enc-dec attn 에서는 source mask 가 사용된다.
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [12]:
def subsequent_mask(size):
    """Mask out subsequent positions.
    3가지 방법이 있음:
    1) torch.ones(size, size, dtype=torch.uint8).tril().unsqueeze(0)
    2) torch.from_numpy(np.tri(size, dtype=np.uint8)).unsqueeze(0)
    3) 아래 주석처리된 오리지널 코드.
    """
    return torch.ones(size, size, dtype=torch.uint8).tril().unsqueeze(0)
    # 2)
#     mask = np.tri(size, dtype=np.uint8)
#     return torch.from_numpy(mask).unsqueeze(0)
#     attn_shape = (1, size, size)
    # 3)
#     mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
#     return torch.from_numpy(mask) == 0

In [13]:
# 상식적인 그 형태 그대로의 마스크.
subsequent_mask(10)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

## Attention

<img src = http://nlp.seas.harvard.edu/images/the-annotated-transformer_33_0.png width="10%" />

$$\mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V$$

In [14]:
def attention(query, key, value, mask=None, dropout=None):
    """ Compute 'Scaled Dot Product Attention'
    query: [B, Q, d_k]
    key  : [B, K, d_k]
    value: [B, V, d_v]
    또는, [B, H, Q, d_k] 의 형태로 H (n_heads) 가 끼어들 수 있음. 아니 항상 그렇겠네...
    """
    d_k = query.size(-1)
    
    # attention score 계산
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    # masking. 0 인 곳은 -inf 로 채워준다.
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    # attention distribution
    p_attn = F.softmax(scores, dim = -1)
    
    if dropout is not None:
        p_attn = dropout(p_attn)
    
    # return attention outputs & attention distribution
    return torch.matmul(p_attn, value), p_attn 

### Multi-head attention

<img src="http://nlp.seas.harvard.edu/images/the-annotated-transformer_38_0.png" width="20%" />

$$
\mathrm{MultiHead}(Q, K, V) = \mathrm{Concat}(\mathrm{head_1}, ...,
\mathrm{head_h})W^O    \\
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$

In [15]:
class MultiHeadedAttention(nn.Module):
    """
    h = 8
    d_model = 512
    d_k = d_v = 64
    """
    def __init__(self, h, d_model, dropout_rate=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None # attention distribution
        self.dropout = nn.Dropout(p=dropout_rate)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            # Source mask: padding 만 마스킹. [B, 1, N] 으로 들어옴.
            #   모든 단어가 패딩을 제외하면 전부 서로를 볼 수 있음.
            # Target mask: padding + 자기 오른쪽 (이후 정보) 은 안 보도록. [B, N-1, N-1] 로 들어옴.
            #   차례대로 자기가 볼 수 있는 마스크를 의미함. target_mask[b, q, k] => q 가 이 k 를 볼 수 있는가?
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        # QW, KW, VW 계산
        # 원래는 W (W_q, W_k, W_v) 는 [d_model, d_k] 다. 근데 어차피 multi-head 니까 한방에 만드는 것.
        # 그래서 linear projection 후 view/transpose 를 통해 차원을 맞춰준다.
        # query 만 차원 설명하면: [B, h, Q, d_k]. 앞에 두개는 배치와 헤드이므로 무시하면 우리가 원하던 형태인 [Q, d_k] 가 된다!
#         query, key, value = \
#             [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
#              for l, x in zip(self.linears, (query, key, value))]
        # 동일함:
        query = self.linears[0](query).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        key   = self.linears[1]( key ).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        value = self.linears[2](value).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        
        # 2) Apply attention on all the projected vectors in batch. 
        # 계산한 projected Q, K, V 에다가 attention 적용
        # x 는 attention output, self.attn 은 attention distribution.
        # x: [B, h, Q, d_k]
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        # x => [B, Q, h, d_k] => [B, Q, d_model]. 즉, concat(heads)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        
        # 4) last linear projection
        return self.linears[-1](x)

## Position-wise Feed-Forward Networks

In [16]:
# 사실 특별할 것 없는 2-layer FC.
# 단, 입력이 [B, Q, d_model] 로 들어오면 이 3번째 차원에 대한 FC 다.
# Linear module 은 원래 마지막 차원에 대해서 FC 를 수행함.
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # output: logits.
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

## Embedding and Softmax

- 디코더의 마지막에 들어가는 linear softmax 의 linear (Generator) 는 word vector => vocab 으로, `[d_model, vocab]` FC.
- Embedding 은 vocab => word vector 로, `[vocab, d_model]` FC.

따라서 두 weight 는 공유할 수 있다!

In [17]:
# word vector size = d_model
# vocabulary size = vocab
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        # look up table
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        # 재미있는 점은 embedding 할때는 softmax 와 반대로 sqrt(d_model) 을 곱해줌!
        return self.lut(x) * math.sqrt(self.d_model)

In [18]:
# decoder 의 마지막 Linear softmax 함수
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

## Positional Encoding

- Watermark for positional information injection
- 포지션 정보를 넣기 위해 사용함. 각 포지션에 맞는 워드 벡터 크기의 포지션 벡터가 생성되고, 이를 그냥 더함으로써 포지션 정보를 입힌다.

$$PE_{(pos,2i)} = sin(pos / 10000^{2i/d_{\text{model}}}) \\
PE_{(pos,2i+1)} = cos(pos / 10000^{2i/d_{\text{model}}})
$$

- pos 는 단어 위치, 2i / 2i+1 은 단어의 워드 임베딩 차원.
- 즉 query \[Q, d_model\] 이 들어오면 `query[pos, 2i(+1)]` 에 이걸 그대로 박아넣으면 됨.

In [19]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout_rate=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout_rate)
        
        # Compute the positional encodings once in log space.
        # 원래 수식과 다르게 log space 에서 계산한 후 exp 를 씌워줘서 복구.
        # 아마 10000 의 2i/d_model 제곱을 하면 값이 너무 커지는 걸 우려한 것 같음.
        # 근데 어차피 2i/d_model 의 최대값이 1인데 이걸 걱정할 필요가 있나...?
#         pe = torch.zeros(max_len, d_model) # [L, D]
#         position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1) # [L, 1]
#         div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32) * -(np.log(10000.0) / d_model)) # [D]
#         # position * div_term = [L, D] (boradcasting)
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0) # [1, L, D]; expand dim for batch dim
        
        
        pe = torch.empty(max_len, d_model) # [L, D]
        pos = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1) # [L, 1]
        indices = torch.arange(0, d_model, 2, dtype=torch.float32) # [D / 2]
        div = 10000. ** (indices / d_model)
        pe[:, 0::2] = torch.sin(pos / div)
        pe[:, 1::2] = torch.cos(pos / div)
        pe.unsqueeze_(0).requires_grad_(False) #.to(device) # expand batch dim
        
        # register_buffer 를 통해 이 module 의 persistent state (no parameter) 로 등록할 수 있다.
        # BN 의 running mean / var 같은 값을 이걸로 등록. 파라메터는 아니라서 학습이 되는 건 아니지만
        # 값을 계속 유지하고, 저장할때도 저장해줘야 하는 등 그라디언트를 계산하지 않는 것만 빼면 파라메터와 동일하게 취급된다.
        self.register_buffer('pe', pe)

    def forward(self, x):
        # No-gradient flow. 어차피 variable 이 없어서 상관 없을 것 같기는 한데.
#         x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [20]:
plt.figure(figsize=(15, 5))
pe = PositionalEncoding(20, 0)
pe.to(device)
y = pe.forward(torch.zeros(1, 100, 20).to(device))
plt.plot(np.arange(100), y[0, :, 4:8].cpu().numpy())
plt.legend(["dim %d"%p for p in [4,5,6,7]])

RuntimeError: CUDA error: out of memory

## Full model

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N), # [self-attn, ff] * 6 스택.
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N), # [self-attn, encdec-attn, ff] * 6 스택.
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)), # src_vocab => wv embedding + pos
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)), # tgt_vocab => wv embedding + pos
        Generator(d_model, tgt_vocab) # Last linear softmax
    )
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1: # weight 인지 체크
            nn.init.xavier_uniform_(p)
            
    return model

# Training

## Synthetic data

- Data copy task
- 그냥 입력으로 들어온 데이터를 카피해서 출력하는 것을 학습

In [ ]:
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
#         src = Variable(data, requires_grad=False)
#         tgt = Variable(data, requires_grad=False)
        src = data.to(device)
        tgt = data.to(device)
        yield Batch(src, tgt, 0)

## Batching

In [ ]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        """
        src: source sentence. word index 형태로 들어오는 듯.
        trg: target sentence. 마찬가지로 word index 형태.
             트레이닝 시에는 이전 아웃풋이 아니라 "true target" word 를 입력으로 넣어줌.
             이를 위해 decoder input trg / decoder target output trg_y 로 나눈 것.
        pad: padding index. masking 을 할 때 padding 은 무조건 못 보게 0으로 마스킹.
             생각해보면 당연한데, padding 이라는 건 문장이 끝났다는 것임. 이건 안 보는게 맞음.
        """
        self.src = src
        # src: [B, N] (N: 단어 개수)
        # src_mask: [B, 1, N]
        self.src_mask = (src != pad).unsqueeze(-2) # 뒤에서 두번째에 차원을 하나 추가
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).sum(dtype=torch.float32) # trg_y 의 총 단어 개수
    
    @staticmethod
    def make_std_mask(tgt, pad):
        """Create a mask to hide padding and future words.
        subsequent_mask() 로 마스킹을 하는데 단: padding 도 못 보게 마스킹.
        """
        tgt_mask = (tgt != pad).unsqueeze(-2) # [B, 1, N]
#         tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        # subsequent_mask: [1, N, N]
        ss_mask = subsequent_mask(tgt.size(-1)).type_as(tgt_mask)
        tgt_mask = tgt_mask & ss_mask
        return tgt_mask

## Training Data and Batching

In [ ]:
# skip
# Synthetic data 에는 필요없는 듯

## Optimizer

- Adam with 
    - beta1 = 0.9
    - beta2 = 0.98
    - eps = 1e-9
- LR decay
    - $ lrate = d_{\text{model}}^{-0.5} \cdot \min({step\_num}^{-0.5},~{step\_num} \cdot {warmup\_steps}^{-1.5}) $

In [ ]:
# 왜 이 옵티마이저 이름이 Noam인가? 모르겠음. 논문에 딱히 그런 얘기 없는 것 같은데.
# 그냥 optimizer + lr control => adam + warmup + lr decay
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        lrate = self.model_size ** (-0.5) * min(step ** (-0.5), step * self.warmup ** (-1.5))
        return self.factor * lrate
        
def get_std_opt(model):
    adam_opt = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
    return NoamOpt(model.src_embed[0].d_model, 2, 4000, adam_opt)

In [ ]:
# Learning rate schedule plot
# Three settings of the lrate hyperparameters.
opts = [NoamOpt(512, 1, 4000, None), 
        NoamOpt(512, 1, 8000, None),
        NoamOpt(256, 1, 4000, None)]
plt.plot(np.arange(1, 20000), [[opt.rate(i) for opt in opts] for i in range(1, 20000)])
plt.legend(["512:4000", "512:8000", "256:4000"])

# Regularization: Label Smoothing

- 논문하곤 좀 다르게 구현되어 있는 듯.
- 논문에선 그냥 e = 0.1 이라는데.

원문 텍스트:

We implement label smoothing using the KL div loss. Instead of using a one-hot target distribution, we create a distribution that has confidence of the correct word and the rest of the smoothing mass distributed throughout the vocabulary.

One-hot target distribution 대신에, smoothing 을 적용한 distribution 을 만들어서 적용. 똑같은 건가?
아래 실험 결과를 보면, `true target = 1 - smoothing` 이 되고 나머지는 `smoothing / N_others` 로 채움.

In [ ]:
class LabelSmoothing(nn.Module):
    """Implement label smoothing.
    이유는 잘 모르겠으나, 여기서 size_average=False 로 주고, 나중에 계산할때 토큰수로 나눠줌.
    패딩 때문에 그런가?
    
    smoothing 자체는 자연스럽게 주는데,
    이걸 KLD 로 계산하면... CE 처럼 계산이 되나?
    참고: https://icim.nims.re.kr/post/easyMath/550
    KLD(y,p) = CE + ylogy. 문제는 ylogy 가 p, 즉 네트워크와 관련 없는 값이므로 최적화할때는 무시할 수 있음.
    따라서 optimize KLD(y,p) = optimize CE(y,p). 단, pytorch 의 CE 는 분포를 입력으로 할 수 없고 
    target class label 만 넣을 수 있어서 스무딩을 적용할 수가 없어서 KLD 를 쓴 것. 
    그냥 직접 곱해서 계산해도 될 것 같은데...;
    """
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        #true_dist = x.clone()
        true_dist = x.new_empty(x.size())
        true_dist.fill_(self.smoothing / (self.size - 2))
        # dim=1 에 대해서, confidence 값을 target index 에 뿌리는 것임.
        true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        # padding_idx 와 같은 target 의 인덱스들을 mask 에 담음
        # 즉, 타겟 자체가 패딩이면 싹다 0으로 채움
        mask = torch.nonzero(target == self.padding_idx)
        if mask.dim() > 1:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        
#         return self.criterion(x, Variable(true_dist, requires_grad=False))
        return self.criterion(x, true_dist)

In [ ]:
# Example of label smoothing.
# 제일 앞이 다 0인건 0번이 패딩이라서 그러함. 패딩에는 smoothing 값을 주지 않는다.
# vocab 에서 패딩 인덱스는 0으로 주고, 아니면 타겟 자체가 패딩이면 싹다 0으로 채움.
crit = LabelSmoothing(5, 0, 0.4)
predict = torch.FloatTensor([[0, 0.2, 0.7, 0.1, 0],
                             [0, 0.2, 0.7, 0.1, 0], 
                             [0, 0.2, 0.7, 0.1, 0]])

v = crit(predict.log(), torch.LongTensor([2, 1, 0]))

crit.true_dist

# Training

- 위에서 언급했듯, training 시에는 이전 timestep 에서 생성한 output 을 auto-regressive 하게 입력으로 넣어주는 것이 아니라, 실제로 나와야하는 true target 값을 입력으로 넣어줌
    - 생각해보면 결국 이전 디코더가 제대로 만들었을 경우에 대해 학습을 해야 하므로 맞는 듯
- 재미있는 것은, **이렇게 되면 decoding phase 가 encoding phase 와 같아진다!** (트레이닝 시)
- 무슨 말이냐면, 디코더에 들어가야 할 인풋을 우리가 처음부터 다 알고있다. 따라서 굳이 앞 timestep 의 결과를 기다릴 필요 없이, 인코더처럼 전체 문장을 한방에 집어넣어서 결과를 뽑아내면 된다.
    - 정확히 말하면 전체 문장에서 제일 앞에는 <BOS> (Beginning of Sentence) 를 넣어주고 제일 마지막 단어는 빼야 함
- 마지막 리니어 소프트맥스는 배치처럼 각 단어에 대해서 다 적용하면 되고.

단, 이건 트레이닝 시 얘기고, 실제 테스트 페이즈에서는 auto-regressive 하게 하나씩 생성해야 한다.

In [ ]:
class SimpleLossCompute:
    """A simple loss compute and train function.
    그냥 decoder output x 와 true label y 를 받아서 criterion 에 따라 로스 계산.
    거기다 backward() 및 opt.step() 도 해줌.
    재미있는 점은 norm - size_average 효과.
    """
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm

## Training Loop

In [ ]:
def run_epoch(data_iter, model, loss_compute):
    """Standard Training and Logging Function
    """
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" % (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
            
    return total_loss / total_tokens

In [ ]:
# Train the simple copy task.
V = 11

criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
# criterion = nn.CrossEntropyLoss(reduction='sum')

model = make_model(V, V, N=2)
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

model.to(device)
criterion.to(device)

for epoch in range(10):
    model.train()
    train_loss = run_epoch(data_gen(V, 30, 20), model, SimpleLossCompute(model.generator, criterion, model_opt))
    
    model.eval()
    eval_loss = run_epoch(data_gen(V, 30, 5), model, SimpleLossCompute(model.generator, criterion, None))
    print(eval_loss.item())

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src).to(device)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src))
        prob = model.generator(out[:, -1]) # all batch, last query (마지막 단어)
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src).fill_(next_word)], dim=1)
    return ys

model.eval()
src = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]).to(device)
src_mask = torch.ones(1, 1, 10).to(device)
print(greedy_decode(model, src, src_mask, max_len=10, start_symbol=1))

In [ ]:
model